In [24]:
import pandas as pd
import geopandas as gpd
import pyodbc
from shapely.geometry import Polygon, MultiPolygon
from shapely.geometry import mapping
import json
from geoalchemy2 import Geometry, WKTElement
import sqlalchemy as sal
from datetime import datetime

In [25]:
districts = gpd.read_file(r'data\WV_Tax_Districts_wmA84\WV_Tax_Districts_wmA84.shp')
districts.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [26]:
server = 'm1data.database.windows.net' 
database = 'm1core.v3' 
username = 'Sayyed_dev' 
password = 'L32IsFgvF20grxRrXZ08' 

In [27]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [28]:
districts.rename(columns={'OBJECTID':'SourceObjectID',
                          'WV_ID':'WVID',
                          'DNAME':'DistrictName',
                          'CNAME':'CountyName',
                          'LABEL':'LabelName',
                          'DNumber':'DistrictID',
                          'CNumber':'CountyID'}, inplace=True)

In [29]:
districts[['DataSource','SourceDateTime','SysStartTime','SysEndTime']] = ''
districts = districts.drop(columns='Meta')
districts = districts.drop(columns='Type')
districts[['SourceObjectID', 'WVID', 'DistrictID', 'CountyID']] = districts[['SourceObjectID', 'WVID', 'DistrictID', 'CountyID']].astype(str)

In [30]:
districts['GeoJSON'] = ''
for index, row in districts.iterrows():
    properties = row.drop('geometry').to_dict()  # Exclude the geometry column
    geometry = mapping(districts['geometry'].iloc[index])  # Convert MultiPolygon to GeoJSON-like dictionary
    feature = {
        "type": "Feature",
        "properties": properties,
        "geometry": geometry
    }
    # Update the 'geojson' column with the GeoJSON string
    districts.at[index, 'GeoJSON'] = json.dumps(feature)

In [31]:
# def wkb_hexer(line):
#     return line.wkb_hex

# # Convert `'geom'` column in GeoDataFrame `gdf` to hex
#     # Note that following this step, the GeoDataFrame is just a regular DataFrame
#     # because it does not have a geometry column anymore. Also note that
#     # it is assumed the `'geom'` column is correctly datatyped.
# districts['geom'] = districts['geometry'].apply(wkb_hexer)

In [32]:
# order = ['SourceObjectID','WVID','DistrictID','DistrictName','CountyID','CountyName','LabelName','GeoJSON','geom','DataSource','SourceDateTime','SysStartTime','SysEndTime','geometry']
# districts = districts[order]
# districts.dtypes

In [33]:
districts.head(1)

,SourceObjectID,WVID,DistrictName,CountyName,DistrictID,CountyID,LabelName,geometry,DataSource,SourceDateTime,SysStartTime,SysEndTime,GeoJSON
0,1,336,Beaver Pond District,Mercer,2,28,Beaver Pond,"MULTIPOLYGON (((-9030113.274 4477713.248, -903...",,,,,"{""type"": ""Feature"", ""properties"": {""SourceObje..."


In [34]:
districts['geom'] = districts['geometry']
districts['geom'] = districts['geom'].astype(str)

In [36]:
order = ['SourceObjectID','WVID','DistrictID','DistrictName','CountyID','CountyName','LabelName','GeoJSON','geom','DataSource','SourceDateTime','SysStartTime','SysEndTime','geometry']
districts = districts[order]
districts.dtypes

SourceObjectID      object
WVID                object
DistrictID          object
DistrictName        object
CountyID            object
CountyName          object
LabelName           object
GeoJSON             object
geom                object
DataSource          object
SourceDateTime      object
SysStartTime        object
SysEndTime          object
geometry          geometry
dtype: object

In [38]:
districts.head(1)

,SourceObjectID,WVID,DistrictID,DistrictName,CountyID,CountyName,LabelName,GeoJSON,geom,DataSource,SourceDateTime,SysStartTime,SysEndTime,geometry
0,1,336,2,Beaver Pond District,28,Mercer,Beaver Pond,"{""type"": ""Feature"", ""properties"": {""SourceObje...",MULTIPOLYGON (((-9030113.274402 4477713.248324...,,,,,"MULTIPOLYGON (((-9030113.274 4477713.248, -903..."


In [37]:
current_datetime = datetime.now()

In [3]:
# for index, row in districts.iterrows():
#     print(index)
#     cursor.execute("INSERT INTO dbo.WVTaxDistricts_Import (SourceObjectID,WVID,DistrictID,DistrictName,CountyID,CountyName,LabelName,GeoJSON,Geom,DataSource,SourceDateTime,SysStartTime,SysEndTime) VALUES (?,?,?,?,?,?,?,?,geometry::STGeomFromText(?, 3857),?,?,?,?)",
#                    row.SourceObjectID, row.WVID, row.DistrictID, row.DistrictName, row.CountyID, row.CountyName, row.LabelName, row.GeoJSON,
#                    row.geom, row.DataSource, row.SourceDateTime, row.SysStartTime, row.SysEndTime)
# cnxn.commit()
# cursor.close()

In [ ]:
for index, row in districts.iterrows():
    print(index)
    cursor.execute("UPDATE dbo.WVTaxDistricts_Import SET SourceObjectID=?, WVID=?, DistrictID=?, DistrictName=?, CountyID=?, CountyName=?, LabelName=?, GeoJSON=?, Geom=geometry::STGeomFromText(?, 3857), DataSource='WV_Tax_Districts_wmA84', SourceDateTime=?, SysStartTime=?, SysEndTime=? WHERE Id=?", 
                   row.SourceObjectID, row.WVID, row.DistrictID, row.DistrictName, row.CountyID, row.CountyName, row.LabelName, row.GeoJSON,
                   row.geom, current_datetime, current_datetime, current_datetime, row.Id)

cnxn.commit()
cursor.close()